<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Managing-2D-masonry-example" data-toc-modified-id="Managing-2D-masonry-example-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Managing 2D masonry example</a></span><ul class="toc-item"><li><span><a href="#Building-example" data-toc-modified-id="Building-example-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Building example</a></span></li><li><span><a href="#Computation" data-toc-modified-id="Computation-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Computation</a></span></li></ul></li><li><span><a href="#3D-example" data-toc-modified-id="3D-example-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>3D example</a></span><ul class="toc-item"><li><span><a href="#Building-example" data-toc-modified-id="Building-example-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Building example</a></span></li><li><span><a href="#Computation" data-toc-modified-id="Computation-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Computation</a></span></li></ul></li></ul></div>

# Managing 2D masonry example

F. Dubois

## Building example

In [ ]:
# basic loadings
from pathlib import Path
import math,copy

import numpy as np

from pylmgc90 import pre

Creating a brick generator

In [ ]:
dim = 2

lx = 0.1
ly = 0.05
brick = pre.brick2D('std_brick', lx, ly)

# what can brick2D generator do ? 
# help(brick)

Creating a line of brick

In [ ]:
nb_bricks = 12
bodies = pre.avatars()
mat = pre.material(name='PLEXx',materialType='RIGID',density=2000.)
mod = pre.model(name='rigid', physics='MECAx', element='Rxx2D', dimension=dim)

for i in range(nb_bricks):
    bodies += brick.rigidBrick(center=[i*lx, ly/2.], model=mod, material=mat, color='BLUEx')

# Draw
pre.visuAvatars(bodies)

Creating a wall with a window (pouah)

In [ ]:
dim = 2

bodies = pre.avatars()
mat = pre.material(name='PLEXx',materialType='RIGID',density=2000.)
mod = pre.model(name='rigid', physics='MECAx', element='Rxx2D', dimension=dim)

# bricks, half-brick, linteau and opening definition
brick      = pre.brick2D('brick',      1.e-1, 5.e-2)
half_brick = pre.brick2D('half-brick', 5.e-2, 5.e-2)
linteau    = pre.brick2D('linteau',    3.e-1, 5.e-2) 
ghost      = pre.brick2D('ghost',      2.e-1, 5.e-2)


# joint thickness
horizontal_joint_thickness = 0.001
vertical_joint_thickness   = 0.001

# rows definitions
even_row        = [half_brick, brick, brick, brick, brick, half_brick]
odd_row         = [brick, brick, brick, brick, brick]
even_window_row = [half_brick, brick, ghost, brick, half_brick]
odd_window_row  = [brick, half_brick, ghost, half_brick, brick]
linteau_row     = [brick, linteau, brick]

# wall as a list of rows
wall = [even_row,
        odd_row ,
        even_row, 
        odd_row ,
        even_window_row, 
        odd_window_row , 
        even_window_row, 
        linteau_row    , 
        even_row       ]


# wall building :
nb_bricks=0
x=0.
y=0.
for j in range(0, len(wall), 1):
    row = wall[j]
    if j % 2 == 0:
        color='BLUEx'
    else:
        color='REDxx'

    x=0.
    for i in range(0, len(row), 1):
        nb_bricks += 1
        brick = row[i]
        if i == 0:
            y += 0.5*brick.ly

        x += 0.5*brick.lx
        if brick.name != 'ghost':
            bodies += brick.rigidBrick(center=[x, y], model=mod, material=mat, color=color)

        x += 0.5*brick.lx + vertical_joint_thickness

    y += 0.5*brick.ly + horizontal_joint_thickness

## wall done... everything else (floor and other container)
mods  = pre.models()
mats  = pre.materials()
svs   = pre.see_tables()
tacts = pre.tact_behavs()

mut = pre.material(name='TDURx',materialType='RIGID',density=2500.)
mats.addMaterial(mat,mut)
mods.addModel(mod)

floor = pre.rigidJonc(axe1=3.e-1, axe2=2.5e-2, center=[2.5e-1, -2.5e-2], 
                      model=mod, material=mut, color='WALLx')
floor.imposeDrivenDof(component=[1, 2, 3],dofty='vlocy')

bodies += floor

try:
    pre.visuAvatars(bodies)
except:
    pass

# interactions management :
lplpl=pre.tact_behav(name='iqsc0',law='IQS_CLB',fric=0.3)
tacts+=lplpl
lpljc=pre.tact_behav(name='iqsc1',law='IQS_CLB',fric=0.5)
tacts+=lpljc
svbbbb = pre.see_table(CorpsCandidat='RBDY2',candidat='POLYG', colorCandidat='BLUEx',behav=lplpl,
                       CorpsAntagoniste='RBDY2', antagoniste='POLYG',colorAntagoniste='BLUEx',alert=5.e-3)
svs+=svbbbb
svbrbr = pre.see_table(CorpsCandidat='RBDY2',candidat='POLYG', colorCandidat='REDxx',behav=lplpl,
                       CorpsAntagoniste='RBDY2', antagoniste='POLYG',colorAntagoniste='REDxx',alert=5.e-3)
svs+=svbrbr
svbbbr = pre.see_table(CorpsCandidat='RBDY2',candidat='POLYG', colorCandidat='BLUEx',behav=lplpl, 
                       CorpsAntagoniste='RBDY2', antagoniste='POLYG',colorAntagoniste='REDxx',alert=5.e-3)
svs+=svbbbr
svpljc = pre.see_table(CorpsCandidat='RBDY2',candidat='POLYG', colorCandidat='BLUEx',behav=lpljc,
                       CorpsAntagoniste='RBDY2', antagoniste='JONCx',colorAntagoniste='WALLx',alert=5.e-3)
svs+=svpljc

# file writing 
datbox = Path('./DATBOX')
datbox.mkdir(exist_ok=True)

In [ ]:
post = pre.postpro_commands()
post.addCommand( pre.postpro_command(name='SOLVER INFORMATIONS', step=1))
post.addCommand( pre.postpro_command(name='VIOLATION EVOLUTION', step=1))

pre.writeDatbox(dim, mats, mods, bodies, tacts, svs, post=post, datbox_path=datbox)

## Computation

In [ ]:
from pylmgc90.chipy import computation

In [ ]:
dim = 2

# time evolution parameters
dt = 1e-3
nb_steps = 100

# theta integrator parameter
theta = 0.5

# nlgs parameters
tol    = 1e-4
relax  = 1.0
norm   = 'Quad '
gs_it1 = 50
gs_it2 = 10
stype  = 'Stored_Delassus_Loops         '

# write parameter
freq_write   = 10

# display parameters
freq_display = 10

In [ ]:
computation.initialize(dim, dt, theta)
for k in range( 1, nb_steps+1 ):
    if k%50 == 0:
        print( f"computing step {k}" )
    computation.one_step(stype, norm, tol ,relax, gs_it1, gs_it2, freq_write, freq_display)
computation.finalize()

# 3D example

## Building example

In [ ]:
# basic loadings
from pathlib import Path
import math,copy

import numpy as np

from pylmgc90 import pre

In [ ]:
mods  = pre.models()
mats  = pre.materials()
svs   = pre.see_tables()
tacts = pre.tact_behavs()

Creating a 3D brick generator

In [ ]:
dim = 3
lx=0.22
ly=0.11
lz=0.06
mat = pre.material(name='PLEXx',materialType='RIGID',density=2000.)
mats+=mat
mod = pre.model(name='rigid', physics='MECAx', element='Rxx3D', dimension=dim)
mods+=mod


simple_brick = pre.brick3D(name='brique simple', lx=lx, ly=ly, lz=lz)

Creating a wall with a builtin function

In [ ]:
# possible function
# print help(pre.paneresse_simple)
# print help(pre.paneresse_double)

In [ ]:
wall   = pre.paneresse_simple(brick_ref=simple_brick, disposition="paneresse")
wall.setFirstRowByLength(first_brick_type="1/2", length=10*lx, joint_thickness=0.0)
wall.setNumberOfRows(10.)
wall.setJointThicknessBetweenRows(0.01)
wall.computeHeight()

bodies = wall.buildRigidWall(origin=[0., 0., 0.], model=mod, material=mat, colors=['BLUEx', 'REDxx'])

# Draw
pre.visuAvatars(bodies)

In [ ]:
bodies1 = wall.buildRigidWallWithoutHalfBricks(origin=[0., 0., 0.], model=mod, material=mat, colors=['BLUEx', 'REDxx'])

# Draw
pre.visuAvatars(bodies1)

Da Silva le maçon

In [ ]:
bodies2=copy.deepcopy(bodies1)

#
bodies2.translate(dy=9.5*lx)
#bodies1.addAvatars(bodies2)
for body in bodies2:
    bodies1+=body

wall.setFirstRowByLength(first_brick_type="1", length=10*lx, joint_thickness=0.0)    
    
bodies3=wall.buildRigidWallWithoutHalfBricks(origin=[0., 0., 0.], model=mod, material=mat, colors=['BLUEx', 'REDxx'])
bodies4=copy.deepcopy(bodies3)
bodies4.translate(dy=9.5*lx)

bodies3.rotate(description='axis', center=np.array([5*lx,5*lx,0.]), axis=[0.,0.,1.], alpha=math.pi/2.)
bodies4.rotate(description='axis', center=np.array([5*lx,5*lx,0.]), axis=[0.,0.,1.], alpha=math.pi/2.)

bodies1 += bodies3
bodies1 += bodies4

#for body in bodies3:
#    bodies1+=body
     
pre.visuAvatars(bodies1)

In [ ]:
# definitions des interactions
#  * loi d'interaction :
lprpr=pre.tact_behav('iqsG0', 'IQS_CLB_g0', fric=0.3)
tacts+=lprpr

#  * table de visibilite :
sv1 = pre.see_table(CorpsCandidat   ='RBDY3', candidat   ='POLYR', colorCandidat   ='BLUEx',
                    CorpsAntagoniste='RBDY3', antagoniste='POLYR', colorAntagoniste='BLUEx',
                     behav='iqsG0', alert=2.5e-2)
svs+=sv1
#  * table de visibilite :
sv2 = pre.see_table(CorpsCandidat   ='RBDY3', candidat   ='POLYR', colorCandidat   ='BLUEx',
                    CorpsAntagoniste='RBDY3', antagoniste='POLYR', colorAntagoniste='REDxx',
                     behav='iqsG0', alert=2.5e-2)
svs+=sv2

In [ ]:
post = pre.postpro_commands()
post.addCommand(pre.postpro_command(name='SOLVER INFORMATIONS', step=1))
post.addCommand(pre.postpro_command(name='VIOLATION EVOLUTION', step=1))

pre.writeDatbox(dim, mats, mods, bodies1, tacts, svs, post=post, datbox_path=datbox)

## Computation

In [ ]:
from pylmgc90.chipy import computation

In [ ]:
dim = 3

# time evolution parameters
dt = 1e-3
nb_steps = 100

# theta integrator parameter
theta = 0.5

# nlgs parameters
tol    = 1e-4
relax  = 1.0
norm   = 'Quad '
gs_it1 = 50
gs_it2 = 50
stype  = 'Stored_Delassus_Loops         '

# write parameter
freq_write   = 10

# display parameters
freq_display = 10

In [ ]:
from pylmgc90 import chipy

chipy.PRPRx_UseCpF2fExplicitDetection(1e-1)
chipy.nlgs_3D_DiagonalResolution()

computation.initialize(dim, dt, theta)
for k in range( 1, nb_steps+1 ):
    if k%50 == 0:
        print( f"computing step {k}" )
    computation.one_step(stype, norm, tol ,relax, gs_it1, gs_it2, freq_write, freq_display)
computation.finalize()